# Análise de Embeddings de Poemas de Clarice Lispector e Medida de Criatividade

Este notebook coleta poemas curtos de Clarice Lispector via OpenAI API, gera embeddings usando diferentes modelos e reduz dimensões para visualizar a similaridade entre os poemas. Além disso, propõe uma medida de criatividade baseada na variabilidade semântica entre as linhas de cada poema.

## 1. Configuração e importações

In [1]:

import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
from main import EmbeddingVisualizer
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics.pairwise import cosine_distances


/Users/joaomadruga/.pyenv/versions/default/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Carregamento de poemas do arquivo JSON

In [2]:
import json
with open('clarice_lispector_poemas.json', encoding='utf-8') as f:
    data = json.load(f)
poemas = {item['name']: item['poem'] for item in data['poems']}
print(f"Carregados {len(poemas)} poemas.")


Carregados 16 poemas.


## 3. Geração de embeddings e redução de dimensionalidade

In [3]:

# Instancia o visualizador de embeddings
visualizer = EmbeddingVisualizer()
model_name = "MiniLM-L6"
# Embeddings dos poemas completos
titulos = list(poemas.keys())
textos = list(poemas.values())
embeddings = visualizer.compute_embeddings(textos, model_name)
# Redução para 2D com PCA
coords = visualizer.reduce_dimensions(embeddings, method="pca", n_components=2)
df = pd.DataFrame(coords, columns=["x", "y"])
df["Título"] = titulos
fig = px.scatter(df, x="x", y="y", text="Título", title="PCA dos poemas de Clarice Lispector")
fig.update_traces(textposition="top center")
fig.show()


/Users/joaomadruga/Desktop/projects/embedding-visualizer/main.py:36: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  OpenAIEmbeddings(
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias

## 4. Medida de criatividade baseada na variabilidade semântica

In [4]:

scores = {}
for titulo, texto in poemas.items():
    linhas = [l for l in texto.splitlines() if l.strip()]
    if len(linhas) < 2:
        scores[titulo] = 0.0
        continue
    emb_linhas = visualizer.compute_embeddings(linhas, model_name)
    dists = cosine_distances(emb_linhas)
    n = len(linhas)
    valores = dists[np.triu_indices(n, 1)]
    scores[titulo] = float(np.mean(valores))
scores_df = pd.DataFrame({"Título": list(scores.keys()), "Criatividade": list(scores.values())})
scores_df = scores_df.sort_values("Criatividade", ascending=False)
fig2 = px.bar(scores_df, x="Título", y="Criatividade", title="Medida de criatividade dos poemas")
fig2.show()


In [5]:
# Métricas adicionais de criatividade
# Métrica 1: Distância Sequencial Média - distância semântica média entre linhas consecutivas de um poema.
dist_seq_scores = {}
for titulo, texto in poemas.items():
    linhas = [l for l in texto.splitlines() if l.strip()]
    if len(linhas) < 2:
        dist_seq_scores[titulo] = 0.0
    else:
        emb_linhas = visualizer.compute_embeddings(linhas, model_name)
        seq_dists = [cosine_distances(emb_linhas[i:i+1], emb_linhas[i+1:i+2])[0][0] for i in range(len(linhas)-1)]
        dist_seq_scores[titulo] = float(np.mean(seq_dists))

# Métrica 2: Diversidade Lexical - proporção de palavras únicas em relação ao total de palavras, indicando originalidade lexical.
lexical_diversity_scores = {}
for titulo, texto in poemas.items():
    palavras = [w for line in texto.splitlines() for w in line.split()]
    if len(palavras) == 0:
        lexical_diversity_scores[titulo] = 0.0
    else:
        lexical_diversity_scores[titulo] = float(len(set(palavras)) / len(palavras))

# Métrica 3: Complexidade Sintática - média de palavras por linha, sugerindo complexidade estrutural.
syntactic_complexity_scores = {}
for titulo, texto in poemas.items():
    linhas = [l for l in texto.splitlines() if l.strip()]
    if len(linhas) == 0:
        syntactic_complexity_scores[titulo] = 0.0
    else:
        syntactic_complexity_scores[titulo] = float(np.mean([len(l.split()) for l in linhas]))

metricas_df = pd.DataFrame({
    "Título": list(poemas.keys()),
    "Distância Sequencial Média": [dist_seq_scores[t] for t in poemas.keys()],
    "Diversidade Lexical": [lexical_diversity_scores[t] for t in poemas.keys()],
    "Complexidade Sintática": [syntactic_complexity_scores[t] for t in poemas.keys()]
})
metricas_df


,Título,Distância Sequencial Média,Diversidade Lexical,Complexidade Sintática
0,Alma luz,0.430193,0.584615,5.909091
1,A lucidez perigosa,0.540589,0.770992,5.038462
2,A perfeição,0.493430,0.616279,6.142857
3,Dá-me a tua mão,0.543674,0.588235,6.263158
4,Eu,0.547932,0.797872,4.700000
5,Mão,0.460978,0.750000,6.500000
6,Mas há a vida,0.571751,0.851852,3.857143
7,"Meu Deus, me dê a coragem",0.492913,0.565517,6.904762
8,Minha alma tem o peso da luz,0.372885,0.600000,6.500000
9,Não te amo mais,0.554692,0.831169,4.812500


In [6]:
# Métricas de criatividade baseadas em embeddings (clusters e dispersão)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
# Computa métricas adicionais usando embeddings diretamente
embed_silhouette = {}
embed_mean_centroid = {}
embed_max_pairwise = {}
for titulo, texto in poemas.items():
    linhas = [l for l in texto.splitlines() if l.strip()]
    if len(linhas) < 2:
        embed_silhouette[titulo] = 0.0
        embed_mean_centroid[titulo] = 0.0
        embed_max_pairwise[titulo] = 0.0
    else:
        embs = visualizer.compute_embeddings(linhas, model_name)
        # Silhueta de Cluster
        n_clusters = 2 if len(linhas) >= 3 else 2
        labels = KMeans(n_clusters=n_clusters, random_state=0).fit_predict(embs)
        embed_silhouette[titulo] = float(silhouette_score(embs, labels))
        # Distância Média ao Centroide
        centroid = embs.mean(axis=0, keepdims=True)
        dists_cent = cosine_distances(embs, centroid).flatten()
        embed_mean_centroid[titulo] = float(dists_cent.mean())
        # Mudança Semântica Máxima
        dists = cosine_distances(embs)
        i,j = np.triu_indices_from(dists, k=1)
        embed_max_pairwise[titulo] = float(dists[i,j].max())

metricas_embed_df = pd.DataFrame({
    'Título': list(poemas.keys()),
    'Silhueta de Cluster': [embed_silhouette[t] for t in poemas.keys()],
    'Distância Média ao Centroide': [embed_mean_centroid[t] for t in poemas.keys()],
    'Mudança Semântica Máxima': [embed_max_pairwise[t] for t in poemas.keys()]
})
metricas_embed_df


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


,Título,Silhueta de Cluster,Distância Média ao Centroide,Mudança Semântica Máxima
0,Alma luz,0.157659,0.238856,0.670840
1,A lucidez perigosa,0.038862,0.340379,0.922237
2,A perfeição,0.054295,0.296115,0.755444
3,Dá-me a tua mão,0.026599,0.328719,0.810112
4,Eu,0.018774,0.326476,0.760088
5,Mão,0.014807,0.251123,0.643918
6,Mas há a vida,0.086972,0.294222,0.828436
7,"Meu Deus, me dê a coragem",0.044994,0.284506,0.730720
8,Minha alma tem o peso da luz,0.072642,0.207510,0.621746
9,Não te amo mais,0.053910,0.299302,0.710117


In [7]:
# Visualização de clusters semânticos por poema (PCA 2D)
from sklearn.decomposition import PCA
cluster_plots = []
for titulo, texto in poemas.items():
    linhas = [l for l in texto.splitlines() if l.strip()]
    if len(linhas) < 2:
        continue
    embs = visualizer.compute_embeddings(linhas, model_name)
    coords = PCA(n_components=2).fit_transform(embs)
    labels = KMeans(n_clusters=2, random_state=0).fit_predict(embs)
    df_plot = pd.DataFrame(coords, columns=['x','y'])
    df_plot['linha'] = linhas
    df_plot['cluster'] = labels.astype(str)
    df_plot['poema'] = titulo
    cluster_plots.append(df_plot)
df_all = pd.concat(cluster_plots, ignore_index=True)
fig = px.scatter(
    df_all, x='x', y='y', color='cluster', facet_col='poema',
    title='Clusters semânticos de linhas por poema (PCA 2D)'
)
fig.update_layout(height=800, width=1200)
fig.show()


In [9]:
# Scatter geral dos clusters semânticos (todas as linhas de todos os poemas)
fig2 = px.scatter(
    df_all, x='x', y='y', color='poema', symbol='cluster',
    title='Visão geral dos clusters semânticos (todas as linhas)'
)
fig2.update_layout(height=600, width=800)
fig2.show()


## 5. Conclusões (em Português)
Observamos que a redução de dimensionalidade mostrou agrupamentos de poemas semanticamente próximos. A medida de criatividade, definida como a média das distâncias semânticas entre as linhas de cada poema, indica que poemas com maior variabilidade interna são potencialmente mais criativos. No entanto, essa métrica serve apenas como um indicador inicial e deve ser refinada com análises qualitativas adicionais e validação com especialistas em literatura.